In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np

In [2]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data


,No.,Time,Source,Destination,Protocol,Length,Topic,Message Identifier,Source Port,Destination Port,Info
0,1238,96.923184,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,41817,1883,Connect Command
1,1239,96.924491,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,42513,1883,Connect Command
2,1240,96.924877,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,38983,1883,Connect Command
3,1241,96.925164,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,37737,1883,Connect Command
4,1242,96.925451,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,60773,1883,Connect Command
...,...,...,...,...,...,...,...,...,...,...,...
2045,5310,197.558531,192.168.15.4,192.168.15.3,MQTT,348,Deviceb'Device - 7',100.0,46211,1883,Publish Message (id=100) [Deviceb'Device - 7']
2046,5311,197.558707,192.168.15.4,192.168.15.3,MQTT,348,Deviceb'Device - 3',101.0,38983,1883,Publish Message (id=101) [Deviceb'Device - 3']
2047,5312,197.560673,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,49821,Publish Ack (id=100)
2048,5314,197.561129,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,46211,Publish Ack (id=100)


In [3]:
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [4]:
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [5]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 2.74760024752544
(Device-1)Median Round Trip Time = 2.7590409999902477
(Device-1)Standard deviation  Round Trip Time = 0.4371288832842367
(Device-1)Min Round Trip Time = 1.288543000015352
(Device-1)Max Round Trip Time = 5.160437000000684
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 2.692814108912097
(Device-2)Median Round Trip Time = 2.730727999988858
(Device-2)Standard deviation  Round Trip Time = 0.5136872497851473
(Device-2)Min Round Trip Time = 1.3099239999974088
(Device-2)Max Round Trip Time = 5.258811000004471
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 2.7496151485140086
(Device-3)Median Round Trip Time = 2.7380009999831145
(Device-3)Standard deviation  Round Trip Time = 0.6100913247428806
(Device-3)Min Round Trip Time = 1.3095529999986866
(Device-3)Max Round Trip Time = 5.724230999987867
-------------------------------------------------------

## Medidas de Tendência Central Geral

In [6]:
from statistics import mean 

print("Mean Round Trip Time = {}".format(mean(data_mean)))
print("Median Round Trip Time = {}".format(mean(data_median)))
print("Standard deviation  Round Trip Time = {}".format(mean(data_std)))
print("Min Round Trip Time = {}".format(mean(data_min)))
print("Max Round Trip Time = {}".format(mean(data_max)))


Mean Round Trip Time = 2.742631584643819
Median Round Trip Time = 2.742631584643819
Standard deviation  Round Trip Time = 2.742631584643819
Min Round Trip Time = 2.742631584643819
Max Round Trip Time = 2.742631584643819
